Age, Chest Pain, Cholestrol, Resting Blood Pressure


In [10]:
library(tidyverse)
library(tidymodels)
library(repr)

In [11]:
switzerland_data<-read.csv("https://raw.githubusercontent.com/byuly/dsci-100-group-project/main/processed.switzerland.data")
